# 인기급상승 채널에 한해 Crawling 진행

In [19]:
# scraping에 필요한 라이브러리 호출
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from tqdm import tqdm
import re
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action = 'ignore')

video{

    category         : str   (영상분류)
    id (pk)          : int   (영상 고유키)
    title            : str   (제목)
    thumbnail        : str   (썸네일 url)
    count_of_views   : int   (조회수)
    comments         : list  (샘플댓글리스트)


    metadata{
        url              : str   (영상url)
        count_of_comment : int   (전체댓글수)
        scrap_count      : int   (샘플댓글수)
    }
}

# 기본 세팅

In [37]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
url = "https://www.youtube.com/feed/trending"
options = webdriver.ChromeOptions()
# 크롬창 띄우기 X
# options.add_argument('headless')
options.add_argument('window-size=1920*1080')
# 크롤링 성능 향상
options.add_argument('--disable-gpu')
options.add_argument('lang=ko_KR, en_US')
options.add_argument('user-agent=' + user_agent)

# 최신 - URL

In [93]:
def recent_check_url():
    driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = options)
    driver.get(url)
    driver.implicitly_wait(3)
    
    # 페이지 스크롤 다운 (모든 동영상 로딩)
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        time.sleep(1.5)
        # 팝업 제거
        try:
            driver.find_element_by_css_selector('#dismiss-button > a').click()

        except:
            pass
    
    # 모든 동영상 (쇼츠 제외) 링크, 제목 수집
    video_links = []
    title_list = []
    for video in driver.find_elements(By.XPATH, "//a[@id='video-title']"):
        if video.get_attribute is not None and 'shorts' not in video.get_attribute('href'):
            video_links.append(video.get_attribute("href"))
            title_list.append(video.text.strip())

    video_links = video_links[:50]
    title_list = title_list[:50]
    
    # 썸네일 이미지 가져오기
    thumbnails = []
    for video_link in video_links:
        driver.get(video_link)
        thumbnail = driver.find_element(By.XPATH, "//meta[@property='og:image']")
        thumbnails.append(thumbnail.get_attribute('content'))
    

    thumnails = thumbnails[:50]
    return video_links, title_list, thumbnails

# 음악 - URL

In [94]:
def music_check_url():
    driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = options)
    driver.get(url)
    driver.implicitly_wait(3)
    
    # 카테고리별 클릭 버튼 추가하기 해당 class name 찾고 (영화)
    # 영화 버튼 클릭하기
    music_button = driver.find_element(By.XPATH, '//*[@id="tabsContent"]/tp-yt-paper-tab[2]/div/div[1]')
    ActionChains(driver).click(music_button).perform()

    # 페이지 스크롤 다운 (모든 동영상 로딩)
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        time.sleep(1.5)
        # 팝업 제거
        try:
            driver.find_element_by_css_selector('#dismiss-button > a').click()

        except:
            pass
    
    # 모든 동영상 (쇼츠 제외) 링크, 제목 수집
    video_links = []
    title_list = []
    for video in driver.find_elements(By.XPATH, "//a[@id='video-title']"):
        if video.get_attribute is not None and 'shorts' not in video.get_attribute('href'):
            video_links.append(video.get_attribute("href"))
            title_list.append(video.text.strip())

    # 썸네일 이미지 가져오기
    thumbnails = []
    for video_link in video_links:
        driver.get(video_link)
        thumbnail = driver.find_element(By.XPATH, "//meta[@property='og:image']")
        thumbnails.append(thumbnail.get_attribute('content'))

    return video_links, title_list, thumbnails

# 게임 - URL

In [95]:
def game_check_url():
    driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = options)
    driver.get(url)
    driver.implicitly_wait(3)
    
    # 카테고리별 클릭 버튼 추가하기 해당 class name 찾고 (영화)
    # 영화 버튼 클릭하기
    game_button = driver.find_element(By.XPATH, '//*[@id="tabsContent"]/tp-yt-paper-tab[3]/div/div[1]')
    ActionChains(driver).click(game_button).perform()

    # 페이지 스크롤 다운 (모든 동영상 로딩)
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        time.sleep(1.5)
        # 팝업 제거
        try:
            driver.find_element_by_css_selector('#dismiss-button > a').click()

        except:
            pass
    
    # 모든 동영상 (쇼츠 제외) 링크, 제목 수집
    video_links = []
    title_list = []
    for video in driver.find_elements(By.XPATH, "//a[@id='video-title']"):
        if video.get_attribute is not None and 'shorts' not in video.get_attribute('href'):
            video_links.append(video.get_attribute("href"))
            title_list.append(video.text.strip())

    # 썸네일 이미지 가져오기
    thumbnails = []
    for video_link in video_links:
        driver.get(video_link)
        thumbnail = driver.find_element(By.XPATH, "//meta[@property='og:image']")
        thumbnails.append(thumbnail.get_attribute('content'))
    
    return video_links, title_list, thumbnails

# 영화 - URL

In [96]:
def movie_check_url():
    driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = options)
    driver.get(url)
    driver.implicitly_wait(3)
    
    # 카테고리별 클릭 버튼 추가하기 해당 class name 찾고 (영화)
    # 영화 버튼 클릭하기
    movie_button = driver.find_element(By.XPATH, '//*[@id="tabsContent"]/tp-yt-paper-tab[4]/div/div[1]')
    ActionChains(driver).click(movie_button).perform()


    # 페이지 스크롤 다운 (모든 동영상 로딩)
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        time.sleep(1.5)
        # 팝업 제거
        try:
            driver.find_element_by_css_selector('#dismiss-button > a').click()

        except:
            pass
    
    # 모든 동영상 (쇼츠 제외) 링크, 제목 수집
    video_links = []
    title_list = []
    for video in driver.find_elements(By.XPATH, "//a[@id='video-title']"):
        if video.get_attribute is not None and 'shorts' not in video.get_attribute('href'):
            video_links.append(video.get_attribute("href"))
            title_list.append(video.text.strip())


    # 썸네일 이미지 가져오기
    thumbnails = []
    for video_link in video_links:
        driver.get(video_link)
        thumbnail = driver.find_element(By.XPATH, "//meta[@property='og:image']")
        thumbnails.append(thumbnail.get_attribute('content'))
    
    return video_links, title_list, thumbnails

# URL 통합 과정

In [97]:
recent_results = recent_check_url()
recent_urls, recent_thumbnail_urls = recent_results[0], recent_results[2]

In [98]:
music_results = music_check_url()
music_urls, music_thumbnail_urls = music_results[0], music_results[2]

In [99]:
game_results = game_check_url()
game_urls, game_thumbnail_urls = game_results[0], game_results[2]

In [100]:
movie_results = movie_check_url()
movie_urls, movie_thumbnail_urls = movie_results[0], movie_results[2]

In [107]:
URLS = [recent_urls] + [music_urls] + [game_urls] + [movie_urls]
THUMBNAILS_URLS = [recent_thumbnail_urls] + [music_thumbnail_urls] + [game_thumbnail_urls] + [movie_thumbnail_urls]

In [112]:
df = pd.DataFrame(columns = ['urls', 'thumbnails_urls'])
df['urls'] = URLS
df['thumbnails_urls'] = THUBNAILS_URLS

In [114]:
df.to_csv('URL.csv',index=False)